In [1]:
from __future__ import print_function

import os
import sys
import numpy as np

from keras.optimizers import SGD
from keras.callbacks import CSVLogger, ModelCheckpoint

sys.path.append(os.path.join(os.getcwd(), os.pardir))

import config

from utils.dataset.data_generator import DataGenerator
from models.vgg_simplified_10 import VGG10

Using Theano backend.
DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpfnuir4/265abc51f7c376c224983485238ff1a5.lib and object C:/Users/Tanuj/AppData/Local/Theano/compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-2.7.12-64/tmpfnuir4/265abc51f7c376c224983485238ff1a5.exp

Using gpu device 0: GeForce GTX 970M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5005)


In [2]:
lr=0.01
n_epochs=500
batch_size=32
input_shape=(140, 140, 3)

name = 'vgg10_140_rgb_lr_%f' % lr

In [3]:
print('loading model...')
model = cnn_w_normalization(input_shape=input_shape)
model.summary()

optimizer = SGD(lr=lr)

print('compiling model...')
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print('done.')

csv_logger = CSVLogger('%s_training.log' % name)
best_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_best.hdf5" % name), verbose=1,
                                          save_best_only=True)

current_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_best.hdf5" % name), verbose=0)

loading model...


NameError: name 'cnn_w_normalization' is not defined

In [ ]:
print('Initializing data generators...')
train_data_gen = DataGenerator(dataset_file=config.train_data_file, batch_size=batch_size)
validation_data_gen = DataGenerator(dataset_file=config.validation_data_file, batch_size=batch_size)
test_data_gen = DataGenerator(dataset_file=config.test_data_file, batch_size=batch_size)
print('done.')

In [ ]:
print('Fitting model...')
history = model.fit_generator(train_data_gen,
                              nb_epoch=n_epochs,
                              samples_per_epoch=train_data_gen.n_batches * batch_size,
                              validation_data=validation_data_gen,
                              nb_val_samples=validation_data_gen.n_samples,
                              verbose=1,
                              callbacks=[csv_logger, best_model_checkpointer, current_model_checkpointer])
print('done.')

In [ ]:
print('Evaluating model...')
score = model.evaluate_generator(test_data_gen, val_samples=test_data_gen.n_samples)
print('done.')

print('Test score:', score[0])
print('Test accuracy:', score[1])